# Criação da sessão do PySpark

In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b880ff5a-d224-4792-9539-ecef15ff5839;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 414ms :: artifacts dl 15ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

# Leitura dos datasets com as métricas dos clientes

In [2]:
BUCKET = 's3a://andre-sprint03-sptech-bucket-cliente/'

In [3]:
df_colera_mapa_calor = spark.read.option('delimiter', ',') \
    .option('header', 'true') \
    .csv(BUCKET + 'df_colera_mapa_calor')

22/06/12 19:48:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
df_colera_regiao = spark.read.option('delimiter', ',') \
    .option('header', 'true') \
    .csv(BUCKET + 'df_colera_regiao')

In [5]:
df_colera_globais = spark.read.option('delimiter', ',') \
    .option('header', 'true') \
    .csv(BUCKET + 'df_colera_globais')

In [6]:
df_agua_tratada_pais_pandas = spark.read.option('delimiter', ',') \
    .option('header', 'true') \
    .csv(BUCKET + 'df_agua_tratada_pais_pandas')

In [7]:
df_populacao_ago_defecando_aberto = spark.read.option('delimiter', ',') \
    .option('header', 'true') \
    .csv(BUCKET + 'df_populacao_ago_defecando_aberto')

In [8]:
df_populacao_br_defecando_aberto = spark.read.option('delimiter', ',') \
    .option('header', 'true') \
    .csv(BUCKET + 'df_populacao_br_defecando_aberto')

In [9]:
from pyspark.sql.types import IntegerType, StringType, DoubleType, DecimalType
from pyspark.sql.functions import col

In [10]:
tbl_colera_mapa_calor = df_colera_mapa_calor.select(
    col('Year').cast(IntegerType()).alias('ANOS'),
    col('Country').cast(StringType()).alias('PAISES'),
    col('Cases').cast(IntegerType()).alias('CASOS'),
    col('GDP (BILLIONS)').cast(DoubleType()).alias('GDP'),
    col('CODE').cast(StringType()).alias('CODIGO')
)

tbl_colera_mapa_calor = tbl_colera_mapa_calor.replace(float('nan'), None)

tbl_colera_mapa_calor.printSchema()

root
 |-- ANOS: integer (nullable = true)
 |-- PAISES: string (nullable = true)
 |-- CASOS: integer (nullable = true)
 |-- GDP: double (nullable = true)
 |-- CODIGO: string (nullable = true)



In [13]:
tbl_colera_regiao = df_colera_regiao.select(
    col('Anos').cast(IntegerType()).alias('Anos'),
    col('Regiao').alias('REGIAO'),
    col('Casos de colera').cast(IntegerType()).alias('CASOS')
)

tbl_colera_regiao.printSchema()

root
 |-- Anos: integer (nullable = true)
 |-- REGIAO: string (nullable = true)
 |-- CASOS: integer (nullable = true)



In [14]:
tbl_colera_globais = df_colera_globais.select(
    col('Anos').cast(IntegerType()).alias('ANOS'),
    col('Casos de colera').cast(IntegerType()).alias('CASOS'),
    col('Mortes por colera').cast(IntegerType()).alias('MORTES')
)

tbl_colera_globais.printSchema()

root
 |-- ANOS: integer (nullable = true)
 |-- CASOS: integer (nullable = true)
 |-- MORTES: integer (nullable = true)



In [15]:
tbl_agua_tratada_pais = df_agua_tratada_pais_pandas.select(
    col('ANOS').cast(IntegerType()),
    col('PAIS'),
    col('CODIGO'),
    col('POPULACAO_USANDO_SERVICOS_BASICOS_AGUA_POTAVEL').cast(IntegerType()).alias('POPULACAO_USANDO_AGUA_POTAVEL')
)

tbl_agua_tratada_pais.printSchema()

root
 |-- ANOS: integer (nullable = true)
 |-- PAIS: string (nullable = true)
 |-- CODIGO: string (nullable = true)
 |-- POPULACAO_USANDO_AGUA_POTAVEL: integer (nullable = true)



In [16]:
df_populacao_ago_defecando_aberto.printSchema()
df_populacao_ago_defecando_aberto.show()

root
 |-- ANOS: string (nullable = true)
 |-- POPULACAO_URBANA_DEFECANDO_EM_ABERTO: string (nullable = true)
 |-- POPULACAO_RURAL_DEFECANDO_EM_ABERTO: string (nullable = true)

+----+------------------------------------+-----------------------------------+
|ANOS|POPULACAO_URBANA_DEFECANDO_EM_ABERTO|POPULACAO_RURAL_DEFECANDO_EM_ABERTO|
+----+------------------------------------+-----------------------------------+
|2000|                                19.1|                              66.71|
|2001|                               18.04|                              66.02|
|2002|                               16.97|                              65.32|
|2003|                                15.9|                              64.62|
|2004|                               14.84|                              63.92|
|2005|                               13.77|                              63.23|
|2006|                                12.7|                              62.53|
|2007|                 

In [17]:
tbl_populacao_ago_defecando_aberto = df_populacao_ago_defecando_aberto.select(
    col('ANOS').cast(IntegerType()),
    col('POPULACAO_URBANA_DEFECANDO_EM_ABERTO').cast(DoubleType()),
    col('POPULACAO_RURAL_DEFECANDO_EM_ABERTO').cast(DoubleType())
)
tbl_populacao_ago_defecando_aberto.printSchema()

root
 |-- ANOS: integer (nullable = true)
 |-- POPULACAO_URBANA_DEFECANDO_EM_ABERTO: double (nullable = true)
 |-- POPULACAO_RURAL_DEFECANDO_EM_ABERTO: double (nullable = true)



In [18]:
tbl_populacao_br_defecando_aberto = df_populacao_br_defecando_aberto.select(
    col('ANOS').cast(IntegerType()),
    col('POPULACAO_URBANA_DEFECANDO_EM_ABERTO').cast(DoubleType()),
    col('POPULACAO_RURAL_DEFECANDO_EM_ABERTO').cast(DoubleType())
)
tbl_populacao_br_defecando_aberto.printSchema()

root
 |-- ANOS: integer (nullable = true)
 |-- POPULACAO_URBANA_DEFECANDO_EM_ABERTO: double (nullable = true)
 |-- POPULACAO_RURAL_DEFECANDO_EM_ABERTO: double (nullable = true)



# Upload dos datasets nas tabelas do Azure

In [19]:
tbl_colera_mapa_calor.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-water-quality.database.windows.net;databaseName=bd_water_quality;') \
 .option('dbtable', 'tbl_colera_mapa_calor') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()

In [20]:
tbl_colera_regiao.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-water-quality.database.windows.net;databaseName=bd_water_quality;') \
 .option('dbtable', 'tbl_colera_regiao') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()

In [21]:
tbl_colera_globais.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-water-quality.database.windows.net;databaseName=bd_water_quality;') \
 .option('dbtable', 'tbl_colera_globais') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()

In [22]:
tbl_agua_tratada_pais.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-water-quality.database.windows.net;databaseName=bd_water_quality;') \
 .option('dbtable', 'tbl_agua_tratada_pais') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()

In [23]:
tbl_populacao_ago_defecando_aberto.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-water-quality.database.windows.net;databaseName=bd_water_quality;') \
 .option('dbtable', 'tbl_populacao_ago_defecando_aberto') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()

In [24]:
tbl_populacao_br_defecando_aberto.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-water-quality.database.windows.net;databaseName=bd_water_quality;') \
 .option('dbtable', 'tbl_populacao_br_defecando_aberto') \
 .option('user', 'urubu100') \
 .option('password', 'Urubu1@@') \
 .save()